In [1]:
import numpy as np
import pandas as pd
import nltk

from sklearn.feature_extraction.text import TfidfVectorizer
import matplotlib.pyplot as plt

nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to /home/nanwang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nanwang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [2]:
df = pd.read_csv('/home/nanwang/github/python-algrothm/python-algrothm/laioffer/notes/watch_reviews.tsv',sep='\t',error_bad_lines=False)

b'Skipping line 8704: expected 15 fields, saw 22\nSkipping line 16933: expected 15 fields, saw 22\nSkipping line 23726: expected 15 fields, saw 22\n'
b'Skipping line 85637: expected 15 fields, saw 22\n'
b'Skipping line 132136: expected 15 fields, saw 22\nSkipping line 158070: expected 15 fields, saw 22\nSkipping line 166007: expected 15 fields, saw 22\nSkipping line 171877: expected 15 fields, saw 22\nSkipping line 177756: expected 15 fields, saw 22\nSkipping line 181773: expected 15 fields, saw 22\nSkipping line 191085: expected 15 fields, saw 22\nSkipping line 196273: expected 15 fields, saw 22\nSkipping line 196331: expected 15 fields, saw 22\n'
b'Skipping line 197000: expected 15 fields, saw 22\nSkipping line 197011: expected 15 fields, saw 22\nSkipping line 197432: expected 15 fields, saw 22\nSkipping line 208016: expected 15 fields, saw 22\nSkipping line 214110: expected 15 fields, saw 22\nSkipping line 244328: expected 15 fields, saw 22\nSkipping line 248519: expected 15 fields,

In [3]:
df.head()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,3653882,R3O9SGZBVQBV76,B00FALQ1ZC,937001370,"Invicta Women's 15150 ""Angel"" 18k Yellow Gold ...",Watches,5,0,0,N,Y,Five Stars,Absolutely love this watch! Get compliments al...,2015-08-31
1,US,14661224,RKH8BNC3L5DLF,B00D3RGO20,484010722,Kenneth Cole New York Women's KC4944 Automatic...,Watches,5,0,0,N,Y,I love thiswatch it keeps time wonderfully,I love this watch it keeps time wonderfully.,2015-08-31
2,US,27324930,R2HLE8WKZSU3NL,B00DKYC7TK,361166390,Ritche 22mm Black Stainless Steel Bracelet Wat...,Watches,2,1,1,N,Y,Two Stars,Scratches,2015-08-31
3,US,7211452,R31U3UH5AZ42LL,B000EQS1JW,958035625,Citizen Men's BM8180-03E Eco-Drive Stainless S...,Watches,5,0,0,N,Y,Five Stars,"It works well on me. However, I found cheaper ...",2015-08-31
4,US,12733322,R2SV659OUJ945Y,B00A6GFD7S,765328221,Orient ER27009B Men's Symphony Automatic Stain...,Watches,4,0,0,N,Y,"Beautiful face, but cheap sounding links",Beautiful watch face. The band looks nice all...,2015-08-31


In [4]:
df.dropna(subset=['review_body'],inplace=True)

In [5]:
df.reset_index(inplace=True, drop=True)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 960056 entries, 0 to 960055
Data columns (total 15 columns):
 #   Column             Non-Null Count   Dtype 
---  ------             --------------   ----- 
 0   marketplace        960056 non-null  object
 1   customer_id        960056 non-null  int64 
 2   review_id          960056 non-null  object
 3   product_id         960056 non-null  object
 4   product_parent     960056 non-null  int64 
 5   product_title      960054 non-null  object
 6   product_category   960056 non-null  object
 7   star_rating        960056 non-null  int64 
 8   helpful_votes      960056 non-null  int64 
 9   total_votes        960056 non-null  int64 
 10  vine               960056 non-null  object
 11  verified_purchase  960056 non-null  object
 12  review_headline    960049 non-null  object
 13  review_body        960056 non-null  object
 14  review_date        960052 non-null  object
dtypes: int64(5), object(10)
memory usage: 109.9+ MB


In [7]:
# use the first 1000 data as our training data
data = df.loc[:999, 'review_body'].tolist()

In [8]:
data

f it was going to work well.<br />I'm  saying this because it was  ridiculous low price for the look that this product can give. surprisingly enough this watch works fine. except for the 24 hr  and montly indicators. they are stationary.  i though the metal part wast going to rust and change color, so far it hasn't .<br />good product for that price.",
 'A little small',
 'Nice',
 'I love it.',
 'excellent',
 'Excellent watch and very fast delivery.  Very pleased',
 "This is a very nice watch. I like to use it when I am working in the garden and in the kitchen. I don't have to worry about having to take it off for dirty work.",
 "This is my second G shock and it's really a cool one. It's a little more expensive than the main line of Gshocks but it more than makes up for it with its appearance. As others have said here, the colors are not pure orange and blue, but a tad washed out. The Japanese special edition has more vibrant color. I knew this before hand and chose this version as I t

In [9]:
# Use nltk's English stopwords.
stopwords = nltk.corpus.stopwords.words('english') 
stopwords.append("n't")
stopwords.append("'s")
stopwords.append("'m")
stopwords.append("br") #html <br>
stopwords.append("watch")

print ("We use " + str(len(stopwords)) + " stop-words from nltk library.")
print (stopwords[:10])

We use 184 stop-words from nltk library.
['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're"]


In [10]:
from nltk.stem.snowball import SnowballStemmer
# from nltk.stem import WordNetLemmatizer 

stemmer = SnowballStemmer("english")

# tokenization and stemming
def tokenization_and_stemming(text):
    tokens = []
    # exclude stop words and tokenize the document, generate a list of string 
    for word in nltk.word_tokenize(text):
        if word.lower() not in stopwords:
            tokens.append(word.lower())

    filtered_tokens = []
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    for token in tokens:
        if token.isalpha():
            filtered_tokens.append(token)
            
    # stemming
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [11]:
tokenization_and_stemming(data[0])

['absolut',
 'love',
 'get',
 'compliment',
 'almost',
 'everi',
 'time',
 'wear',
 'dainti']

In [12]:
data[0]

'Absolutely love this watch! Get compliments almost every time I wear it. Dainty.'

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
# define vectorizer parameters
# TfidfVectorizer will help us to create tf-idf matrix
# max_df : maximum document frequency for the given word
# min_df : minimum document frequency for the given word
# max_features: maximum number of words
# use_idf: if not true, we only calculate tf
# stop_words : built-in stop words
# tokenizer: how to tokenize the document
# ngram_range: (min_value, max_value), eg. (1, 3) means the result will include 1-gram, 2-gram, 3-gram
tfidf_model = TfidfVectorizer(max_df=0.99, max_features=1000,
                                 min_df=0.01, stop_words='english',
                                 use_idf=True, tokenizer=tokenization_and_stemming, ngram_range=(1,1))

tfidf_matrix = tfidf_model.fit_transform(data) #fit the vectorizer to synopses

print ("In total, there are " + str(tfidf_matrix.shape[0]) + \
      " reviews and " + str(tfidf_matrix.shape[1]) + " terms.")

In total, there are 1000 reviews and 239 terms.


In [14]:
tfidf_matrix

<1000x239 sparse matrix of type '<class 'numpy.float64'>'
	with 6891 stored elements in Compressed Sparse Row format>

In [15]:
tfidf_matrix.toarray()

array([[0.       , 0.5125863, 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       ...,
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ],
       [0.       , 0.       , 0.       , ..., 0.       , 0.       ,
        0.       ]])

In [16]:
# words
tf_selected_words = tfidf_model.get_feature_names()

In [17]:
# print out words
tf_selected_words

['abl',
 'absolut',
 'accur',
 'actual',
 'adjust',
 'alarm',
 'alreadi',
 'alway',
 'amaz',
 'amazon',
 'anoth',
 'arm',
 'arriv',
 'automat',
 'awesom',
 'bad',
 'band',
 'batteri',
 'beauti',
 'best',
 'better',
 'big',
 'bit',
 'black',
 'blue',
 'bought',
 'box',
 'bracelet',
 'brand',
 'break',
 'bright',
 'broke',
 'button',
 'buy',
 'ca',
 'came',
 'case',
 'casio',
 'chang',
 'cheap',
 'clasp',
 'classi',
 'clock',
 'color',
 'come',
 'comfort',
 'compliment',
 'cool',
 'cost',
 'crown',
 'crystal',
 'dark',
 'date',
 'daughter',
 'day',
 'deal',
 'definit',
 'deliveri',
 'design',
 'dial',
 'differ',
 'difficult',
 'disappoint',
 'display',
 'dress',
 'durabl',
 'easi',
 'easili',
 'end',
 'everi',
 'everyday',
 'everyth',
 'exact',
 'excel',
 'expect',
 'expens',
 'face',
 'fair',
 'far',
 'fast',
 'featur',
 'feel',
 'fell',
 'fine',
 'finish',
 'fit',
 'function',
 'gave',
 'gift',
 'gold',
 'good',
 'got',
 'great',
 'hand',
 'happi',
 'hard',
 'heavi',
 'high',
 'hold',


In [18]:
# k-means clustering
from sklearn.cluster import KMeans

num_clusters = 5

# number of clusters
km = KMeans(n_clusters=num_clusters)
km.fit(tfidf_matrix)

clusters = km.labels_.tolist()

In [19]:
# create DataFrame films from all of the input files.
product = { 'review': df[:1000].review_body, 'cluster': clusters}
frame = pd.DataFrame(product, columns = ['review', 'cluster'])

In [20]:
frame.head(10)

,review,cluster
0,Absolutely love this watch! Get compliments al...,2
1,I love this watch it keeps time wonderfully.,2
2,Scratches,0
3,"It works well on me. However, I found cheaper ...",0
4,Beautiful watch face. The band looks nice all...,0
5,"i love this watch for my purpose, about the pe...",2
6,"for my wife and she loved it, looks great and ...",4
7,I was about to buy this thinking it was a Swis...,0
8,Watch is perfect. Rugged with the metal &#34;B...,4
9,Great quality and build.<br />The motors are r...,0


In [21]:
print ("Number of reviews included in each cluster:")
frame['cluster'].value_counts().to_frame()

Number of reviews included in each cluster:


,cluster
0,669
2,111
4,81
1,77
3,62


In [22]:
km.cluster_centers_

# 239数的list -> cluster 0的中心点的tf-idf值
#-> assumption: 中心点的值可以代表这个cluster
#-> tf-idf值越大，对应的词越能代表这个document
#-> 选出了tf-idf最大的6个值对应的词来代表这个cluster

array([[0.00563952, 0.00435867, 0.00381727, ..., 0.00651155, 0.01795646,
        0.01502076],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00851693,
        0.        ],
       [0.        , 0.04125958, 0.        , ..., 0.01222159, 0.01717547,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.00784247,
        0.        ],
       [0.00398137, 0.        , 0.        , ..., 0.00275151, 0.        ,
        0.01911375]])

In [23]:
km.cluster_centers_.shape

(5, 239)

In [24]:
print ("<Document clustering result by K-means>")

#km.cluster_centers_ denotes the importances of each items in centroid.
#We need to sort it in decreasing-order and get the top k items.
order_centroids = km.cluster_centers_.argsort()[:, ::-1] 

Cluster_keywords_summary = {}
for i in range(num_clusters):
    print ("Cluster " + str(i) + " words:", end='')
    Cluster_keywords_summary[i] = []
    for ind in order_centroids[i, :6]: #replace 6 with n words per cluster
        Cluster_keywords_summary[i].append(tf_selected_words[ind])
        print (tf_selected_words[ind] + ",", end='')
    print ()
    
    cluster_reviews = frame[frame.cluster==i].review.tolist()
    print ("Cluster " + str(i) + " reviews (" + str(len(cluster_reviews)) + " reviews): ")
    print (", ".join(cluster_reviews))
    print ()

 as screwdrivers, the bolts came out and went back in easily, even though they were badly bent. That's the only downside to this product; it doesn't come with the screws, but it said that in the description. But the band is nice and legit. Nicer than my ratty old watch!, I like it!!, Is so cute .love it, Excellent value for this watch, for smaller wrist - best purchase price you could find anywhere, Strap too short and I don't have large wrists.  Loops keep migrating.  Other than that it does it's job albeit &#34;snugly&#34;.  For 12 bucks I'm being too fussy., Magnifique watch very sturdy very high quality 100% recommend use it everyday, I received this product at a discount in exchange for an honest review.  Fits the wrist very well and it's a cool watch.  Does not come with any instructions and it's pretty tough to set., I like this watch. It's got a retro look to it. I have a skinny wrist so the band is on nearly the tightest notch, but that's ok. The face is big but not overwhelmi

In [25]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [26]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.54749152 0.27216957 0.06006541 0.05948725 0.06078625]
 [0.0857093  0.09031294 0.65596823 0.08373725 0.0842723 ]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000019 0.10000118 0.10005656 0.59994176 0.10000031]
 [0.0672203  0.73105105 0.06689111 0.06800979 0.06682774]
 [0.72985509 0.06743175 0.06792441 0.06737022 0.06741853]]


In [27]:
# Use LDA for clustering
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(n_components=5)

In [28]:
# document topic matrix for tfidf_matrix_lda
lda_output = lda.fit_transform(tfidf_matrix)
print(lda_output.shape)
print(lda_output)

(1000, 5)
[[0.29295114 0.06186259 0.05943308 0.05984604 0.52590716]
 [0.0877571  0.08365425 0.08352753 0.65964709 0.08541403]
 [0.2        0.2        0.2        0.2        0.2       ]
 ...
 [0.10000129 0.10024461 0.59965407 0.1000003  0.10009974]
 [0.4023357  0.39605516 0.06738643 0.06673901 0.06748371]
 [0.29749672 0.06758661 0.06821887 0.49781451 0.06888329]]


In [29]:
# topics and words matrix
topic_word = lda.components_
print(topic_word.shape)
print(topic_word)

(5, 239)
[[3.10651875 7.69098592 0.2028007  ... 3.77848758 9.55927807 0.20230091]
 [0.21480102 0.20074283 0.20391624 ... 0.20315714 0.79022789 1.83273785]
 [0.74009549 0.20038263 0.20039804 ... 0.20035712 0.20097032 0.21939375]
 [0.72505013 0.20055606 1.03938666 ... 0.5083148  2.38676256 2.1762812 ]
 [0.30886169 0.20309538 1.90725344 ... 2.24538065 3.12414896 8.16639176]]


In [30]:
# column names
topic_names = ["Topic" + str(i) for i in range(lda.n_components)]

# index names
doc_names = ["Doc" + str(i) for i in range(len(data))]

df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topic_names, index=doc_names)

# get dominant topic for each document
topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['topic'] = topic

df_document_topic.head(10)

,Topic0,Topic1,Topic2,Topic3,Topic4,topic
Doc0,0.29,0.06,0.06,0.06,0.53,4
Doc1,0.09,0.08,0.08,0.66,0.09,3
Doc2,0.20,0.20,0.20,0.20,0.20,0
Doc3,0.06,0.06,0.06,0.06,0.76,4
Doc4,0.34,0.04,0.04,0.04,0.54,4
Doc5,0.42,0.35,0.07,0.07,0.08,0
Doc6,0.07,0.06,0.06,0.74,0.06,3
Doc7,0.06,0.75,0.06,0.06,0.06,1
Doc8,0.19,0.04,0.04,0.05,0.67,4
Doc9,0.06,0.76,0.06,0.06,0.06,1


In [31]:
df_document_topic['topic'].value_counts().to_frame()

,topic
4,286
0,264
2,184
3,144
1,122


In [32]:
# topic word matrix
print(lda.components_)
# topic-word matrix
df_topic_words = pd.DataFrame(lda.components_)

# column and index
df_topic_words.columns = tfidf_model.get_feature_names()
df_topic_words.index = topic_names

df_topic_words.head()

[[3.10651875 7.69098592 0.2028007  ... 3.77848758 9.55927807 0.20230091]
 [0.21480102 0.20074283 0.20391624 ... 0.20315714 0.79022789 1.83273785]
 [0.74009549 0.20038263 0.20039804 ... 0.20035712 0.20097032 0.21939375]
 [0.72505013 0.20055606 1.03938666 ... 0.5083148  2.38676256 2.1762812 ]
 [0.30886169 0.20309538 1.90725344 ... 2.24538065 3.12414896 8.16639176]]


,abl,absolut,accur,actual,adjust,alarm,alreadi,alway,amaz,amazon,...,weight,went,wife,wind,wish,work,worn,worth,wrist,year
Topic0,3.106519,7.690986,0.202801,0.818268,2.356015,2.698626,0.201529,0.202470,9.960818,0.201567,...,0.203156,0.200473,0.202470,0.205469,0.946679,0.428263,1.595219,3.778488,9.559278,0.202301
Topic1,0.214801,0.200743,0.203916,0.200931,0.202409,1.402577,0.203951,1.613821,0.201364,2.384100,...,0.200211,0.201110,0.201230,1.725259,1.025212,3.603270,0.300744,0.203157,0.790228,1.832738
Topic2,0.740095,0.200383,0.200398,0.200488,0.695155,0.202205,3.015077,0.202811,0.200807,0.222590,...,0.201264,0.200650,0.200422,0.201104,0.200242,27.377240,0.200090,0.200357,0.200970,0.219394
Topic3,0.725050,0.200556,1.039387,1.729957,0.202750,1.428277,0.200266,0.274974,0.200993,0.201588,...,1.190325,0.202343,11.836279,1.876613,0.200635,1.535448,0.200207,0.508315,2.386763,2.176281
Topic4,0.308862,0.203095,1.907253,2.906753,2.836842,0.208726,2.142546,2.545928,0.204852,3.792695,...,3.923407,2.912599,0.201997,0.225264,2.197740,8.117446,1.906833,2.245381,3.124149,8.166392


In [33]:
# print top n keywords for each topic
def print_topic_words(tfidf_model, lda_model, n_words):
    words = np.array(tfidf_model.get_feature_names())
    topic_words = []
    # for each topic, we have words weight
    for topic_words_weights in lda_model.components_:
        top_words = topic_words_weights.argsort()[::-1][:n_words]
        topic_words.append(words.take(top_words))
    return topic_words

topic_keywords = print_topic_words(tfidf_model=tfidf_model, lda_model=lda, n_words=15)        

df_topic_words = pd.DataFrame(topic_keywords)
df_topic_words.columns = ['Word '+str(i) for i in range(df_topic_words.shape[1])]
df_topic_words.index = ['Topic '+str(i) for i in range(df_topic_words.shape[0])]
df_topic_words

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,love,beauti,perfect,look,pretti,nice,band,amaz,size,realli,wrist,gift,small,fit,bought
Topic 1,like,use,excel,buy,look,style,watch,qualiti,strap,big,dial,great,invicta,review,recommend
Topic 2,good,nice,work,product,expect,price,excel,qualiti,seller,deliveri,thank,fast,pleas,week,stop
Topic 3,great,look,wife,love,feel,time,need,bad,deal,cheap,want,money,return,comfort,like
Topic 4,time,band,day,light,awesom,wear,hand,look,month,color,love,batteri,compliment,second,like
